In [ ]:
#%% Initial imports etc
import numpy
from numpy.linalg import norm
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import os
import sys
import shutil

import sirf.pyiutilities as pyiutil

from ccpi.utilities.jupyter import *
from ccpi.utilities.display import *

#%% Use the 'pet' prefix for all STIR-based SIRF functions
# This is done here to explicitly differentiate between SIRF pet functions and 
# anything else.
import sirf.STIR as pet
from sirf.Utilities import examples_data_path

import os
import psutil
process = psutil.Process(os.getpid())

# schemes: file, memory
pet.AcquisitionData.set_storage_scheme('file')

#%% Go to directory with input files
# Adapt this path to your situation (or start everything in the relevant directory)
os.chdir(examples_data_path('PET'))

#%% Copy files to a working folder and change directory to where these files are.
# We do this to avoid cluttering your SIRF files. This way, you can delete 
# working_folder and start from scratch.
if True:
    shutil.rmtree('working_folder/memorytest',True)
    shutil.copytree('brain','working_folder/memorytest')
os.chdir('working_folder/memorytest')

#%% Read in images
# Here we will read some images provided with the demo using the ImageData class.
# These are in Interfile format. (A text header pointing to a .v file with the binary data).
image = pet.ImageData('emission.hv')

#%% Create a SIRF acquisition model
# We will use the ray-tracing matrix here as our simple PET model.
# There is more to the accquisition model, but that's for another demo.
am = pet.AcquisitionModelUsingRayTracingMatrix()
# Ask STIR to use 5 LORs per sinogram-element
am.set_num_tangential_LORs(5)

#%% Specify sinogram dimensions
# We need to say what scanner to use, what dimensions etc.
# You do this by using existing PET data as a 'template'. 
# Here, we read a file supplied with the demo as an AcquisitionData object
templ = pet.AcquisitionData('template_sinogram.hs')
# Now set-up our acquisition model with all information that it needs about the data and image.
am.set_up(templ,image)

In [ ]:
import sirf.Reg as Reg
from scipy.spatial.transform import Rotation as R

def get_resampler(directions, angles, image, degrees=True ):
    '''example input 'zy', [87,13], degrees=True'''
    r = R.from_euler(directions, angles, degrees=degrees)

    mat = r.as_dcm()

    mat4 = numpy.eye(4)
    print (mat4.shape, mat4)

    for i in range(3):
        for j in range(3):
            mat4[i][j] = mat[i][j]

    tm = Reg.AffineTransformation(mat4)

    mat = tm.as_array()

    resampler = Reg.NiftyResample()
    print (id(resampler), resampler.handle)
    resampler.set_reference_image(image)
    resampler.set_floating_image(image)
    resampler.add_transformation(tm)
    resampler.set_padding_value(0)
    resampler.set_interpolation_type_to_linear()
    
    return resampler

In [ ]:
# create different motion state
rotations = [[-1.2,3.],[1.2,-3.],[0.,-5.], [.2,2.]]
rotations = [ [10 * rot[0],rot[1]] for rot in rotations ]

# resamplers = [ get_resampler('zy', rot, image, degrees=True) for rot in rotations ]
resampler = get_resampler('zy', [-12.,30.], image, degrees=True) 
# create the new AcquisitionData for the motion states
memocc = []


def f(image):
    step = resampler.direct(image)
    out = resampler.adjoint(step)
    return out

for i in range(1):
    # new ImageData
    step = resampler.direct(image)
    # new AcquisitionData
    out = resampler.adjoint(step)
#     f(image)
    if i % 100 == 0 and i > 0:
        memocc.append(process.memory_percent())
        print(i, memocc[-1], memocc[-1]/float(i))
# del out


id_old = None
handle_old = None

In [ ]:
for i in range(2):
    out = f(image)
print (resampler.handle)

In [ ]:
ids = []

In [ ]:
import sys

step = resampler.direct(image)
ids.append((id(step), step.handle))
for i in ids:
    print (i)


print (sys.getrefcount(step))
#out = resampler.adjoint (step)
    


In [ ]:
for i, handle in ids:
    print ("trying to delete handle", handle)
    #pyiutil.deleteDataHandle(handle)
    step.__del__()
# out = resampler.adjoint(step)
# print (id (out))
# print (out.handle)

In [ ]:
memocc2 = []
step = am.direct(image)
out = am.adjoint(step)

for i in range(101):
    # new ImageData
    am.direct(image, out=step)
    # new AcquisitionData
    am.adjoint(step, out=out)
    if i % 100 == 0 and i > 0:
        memocc2.append(process.memory_percent())
        print(i, memocc2[-1], (memocc2[-1])/float(i))

In [15]:
import numpy

class A(object):
    def __init__(self,array,parent):
        self.array = array.copy()
        self.parent = parent
        print ("calling __init__ {}".format(self.__class__.__name__))
        
    def __del__(self):
        print ("calling __del__ {}".format(self.__class__.__name__))
        
    def direct(self,x):
        return type(self)(self.array + x, id(self.array))
    
class B(A):
    def __init__(self, array):
        super(B,self).__init__(array)
        self.a = A(array,id(array))

In [18]:
aa = A(numpy.zeros(shape=(10,10)), None)
x = numpy.ones(shape = (10,10))

bb = B(numpy.zeros(shape=(10,10)),1)

calling __init__ A
calling __del__ A
calling __del__ B
calling __del__ B


TypeError: __init__() takes exactly 2 arguments (3 given)

In [19]:
b = aa.direct(x)
c = bb.direct(x)

calling __init__ A
calling __del__ A


TypeError: super(type, obj): obj must be an instance or subtype of type

In [ ]:
class FooType(object):
    def __init__(self, id, parent):
        self.id = id
        self.parent = parent
        print 'Foo', self.id, 'born'

    def __del__(self):
        print 'Foo', self.id, 'died'


class BarType(object):
    def __init__(self, id):
        self.id = id
        self.foo = FooType(id, self)
        print 'Bar', self.id, 'born'

    def __del__(self):
        print 'Bar', self.id, 'died'


b = BarType(12)